# Data

In [1]:
# Pull assets
!git submodule update --init --recursive

## Process MIDI files
Convert each MIDI file into a sequence of events as described in the paper, and saved as a npy file.

### Classical String Quartets

In [1]:
import os
import json
import numpy as np
from tqdm import tqdm
from utilities.midi_io import MIDI
from utilities import mkdir

# Specify the time quantization set and time unit
from models import CSQ_TIME_QUANTIZATION
time_unit = "crotchet"

# Specify data augmentation parameters
# pitch_shifts = [-3, -2, -1, 0, 1, 2, 3]
# time_scales = [0.95, 0.975, 1.0, 1.025, 1.05]

# Uncomment the following lines for no augmentation
pitch_shifts = [0]
time_scales = [1.0]

# Get the list of midi files
index_filename = "aimgef-assets/CSQ/fast_first/index.json"
with open(index_filename, mode="r") as f:
    midi_list = json.load(f)

# Start processing
for midi in tqdm(midi_list):
    valid_set = [1211, 1219, 1240, 1827, 1893, 2322, 2368]
    split = "train" if midi["ID"] not in valid_set else "validation"

    for ps in pitch_shifts:
        for ts in time_scales:
            mid = MIDI(
                time_quantization=CSQ_TIME_QUANTIZATION,
                time_unit=time_unit
            )
            # The dimensions of sequence is [num_tracks, num_tokens]
            # num_tracks is 1 if merge_tracks is True
            sequence = mid.process(
                filename=os.path.join(
                    "aimgef-assets/CSQ/fast_first/midi",
                    f'{midi["ID"]}.mid'
                ),
                ignore_velocity=True,
                merge_tracks=False,
                pitch_shift=ps,
                time_scale=ts
            )
            filename = os.path.join(
                "datasets",
                "CSQ",
                split,
                f'{midi["ID"]}_p[{ps}]_t[{ts}].npy'
            )
            mkdir(filename)
            np.save(filename, np.array(sequence, dtype=int))


100%|██████████| 71/71 [00:18<00:00,  3.90it/s]


### Plot the processed sequence

In [1]:
import numpy as np
from utilities.midi_io import MIDI
from models import CSQ_TIME_QUANTIZATION

mid = MIDI(
    time_quantization=CSQ_TIME_QUANTIZATION,
    time_unit="crotchet",
)
seq = mid.process("scratch/test_midi_like_decoding.mid", ignore_velocity=False, merge_tracks=False, pitch_shift=0, time_scale=1)


mid_rec = MIDI(
    time_quantization=CSQ_TIME_QUANTIZATION,
    time_unit="crotchet",
)

pm = mid_rec.to_midi(seq)
pm.write("scratch/out.mid")
